# TensorFlow Keras ResNet50 Example on Habana Gaudi<sup>TM</sup>

This Jupyter Notebook example demonstrates how to train Keras ResNet50 on Habana Gaudi<sup>TM</sup> device with TensorFlow framework. The neural network is built with Keras APIs, and trained with synthetic data.

In [ ]:
%pwd

In [ ]:
!ls

We will clone Habana `Model-References` repository 0.15.4 branch to the current directory.

In [ ]:
!git clone -b 0.15.4 https://github.com/HabanaAI/Model-References.git

Check if `Model-References` folder shows up in the current directory.

In [ ]:
%ls

Now let's check if `Model-References` repository location is in the sys.path. If not, add it.

In [ ]:
import sys
print(sys.path)

Add `Model-References` location and `resnet_keras` location to the `sys.path` so that the dependent Python packages in `Model-References` repository are loaded for ResNet50 training. 

**NOTE:** The following Python statements assume the `Model-Reference` repository was cloned to $HOME directory. If you clone it to a different location, modify it accordingly.

In [ ]:
sys.path.append('/home/ubuntu/Model-References/TensorFlow/computer_vision/Resnets/resnet_keras')
sys.path.append('/home/ubuntu/Model-References')
print(sys.path)

Now, let's import the common Python packages and tensorflow framework.

In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from absl import app
from absl import flags
from absl import logging

import tensorflow as tf

Then, we will import the dependent Python packages from Habana Model-References repository.

In [ ]:
from TensorFlow.common.modeling import performance
from TensorFlow.common.training import controller
from TensorFlow.common.library_loader import load_habana_module
from TensorFlow.common.debug import dump_callback
from TensorFlow.common.horovod_helpers import synapse_logger_init
from TensorFlow.common.tb_utils import write_hparams_v2

from TensorFlow.utils.logs import logger
from TensorFlow.utils.misc import distribution_utils
from TensorFlow.utils.misc import keras_utils
from TensorFlow.utils.misc import model_helpers

from TensorFlow.computer_vision.common import imagenet_preprocessing
from TensorFlow.computer_vision.Resnets.resnet_keras.local_flags import core as flags_core
from TensorFlow.computer_vision.Resnets.utils.optimizers.keras import lars_util
from TensorFlow.computer_vision.Resnets.resnet_keras import common
from TensorFlow.computer_vision.Resnets.resnet_keras import resnet_runnable
from TensorFlow.computer_vision.Resnets.resnet_keras.common import adjust_batch_size

from central.habana_model_yaml_config import HabanaModelYamlConfig
from central.habana_model_runner_utils import HabanaEnvVariables

Now, let's define the arguments for Keras APIs, Habana APIs, and LARS optimizer APIs.

In [ ]:
logging.set_verbosity(logging.INFO)
# define keras args
common.define_keras_flags()
# define habana args
common.define_habana_flags()
# define LARS args
lars_util.define_lars_flags()

Then we create a function to parse the arguments from yaml configuration file to get user's input.

In [ ]:
def parse_args_yaml_config(config_file):
    yaml_config = HabanaModelYamlConfig('resnet_keras', config_file)
    
    env_args = yaml_config.get_env_vars()
    model_params=yaml_config.get_parameters()
    
    cmd_args = []
    exclude_fields = ['use_horovod', 'num_workers_per_hls', 'hls_type']
    yaml_config.add_parameters_except(cmd_args, exclude_fields)
    
    return env_args, cmd_args

Let's define the functions used for ResNet50 training.

In [ ]:
def build_stats(runnable, time_callback):
    """Normalizes and returns dictionary of stats.

    Args:
      runnable: The module containing all the training and evaluation metrics.
      time_callback: Time tracking callback instance.

    Returns:
      Dictionary of normalized results.
    """
    stats = {}

    if not runnable.flags_obj.skip_eval:
        stats['eval_loss'] = runnable.test_loss.result().numpy()
        stats['eval_acc'] = runnable.test_accuracy.result().numpy()

        stats['train_loss'] = runnable.train_loss.result().numpy()
        stats['train_acc'] = runnable.train_accuracy.result().numpy()

    if time_callback:
        timestamp_log = time_callback.timestamp_log
        stats['step_timestamp_log'] = timestamp_log
        stats['train_finish_time'] = time_callback.train_finish_time
        if time_callback.epoch_runtime_log:
            stats['avg_exp_per_second'] = time_callback.average_examples_per_second

    return stats

In [ ]:
def get_num_train_iterations(flags_obj):
    """Returns the number of training steps, train and test epochs."""
    
    train_steps = (
        imagenet_preprocessing.NUM_IMAGES['train'] // adjust_batch_size(flags_obj.batch_size))
    train_epochs = flags_obj.train_epochs

    if flags_obj.train_steps:
        train_steps = min(flags_obj.train_steps, train_steps)
        train_epochs = 1

    eval_steps = (
        imagenet_preprocessing.NUM_IMAGES['validation'] // flags_obj.batch_size)

    return train_steps, train_epochs, eval_steps


In [ ]:
def _steps_to_run(steps_in_current_epoch, steps_per_epoch, steps_per_loop):
    """Calculates steps to run on device."""
    
    if steps_per_loop <= 0:
        raise ValueError('steps_per_loop should be positive integer.')
    if steps_per_loop == 1:
        return steps_per_loop
    return min(steps_per_loop, steps_per_epoch - steps_in_current_epoch)


Finally, we will define the function to run training.

In [ ]:
def run(flags_obj):
    """Run ResNet50 training with synthetic data and eval loop using custom training loops.

    Args:
      flags_obj: An object containing parsed flag values.

    Raises:
      ValueError: If fp16 is passed as it is not currently supported.

    Returns:
      Dictionary of training and eval stats.
    """
    
    keras_utils.set_session_config(
        enable_eager=flags_obj.enable_eager,
        enable_xla=flags_obj.enable_xla)
    performance.set_mixed_precision_policy(flags_core.get_tf_dtype(flags_obj))

    # set data format
    data_format = flags_obj.data_format
    
    if data_format is None:
        data_format = ('channels_first'
                       if tf.test.is_built_with_cuda() else 'channels_last')
    tf.keras.backend.set_image_data_format(data_format)

    batch_size = flags_obj.batch_size
    model_dir = flags_obj.model_dir

    strategy = distribution_utils.get_distribution_strategy(
        distribution_strategy=flags_obj.distribution_strategy,
        num_gpus=flags_obj.num_gpus,
        all_reduce_alg=flags_obj.all_reduce_alg,
        num_packs=flags_obj.num_packs,
        tpu_address=flags_obj.tpu)
    
    train_writer, eval_writer = None, None
    if flags_obj.enable_tensorboard:
        import os
        train_writer = tf.summary.create_file_writer(model_dir)
        eval_writer = tf.summary.create_file_writer(os.path.join(model_dir, 'eval'))
        write_hparams_v2(train_writer, flags_obj.flag_values_dict())
    

    per_epoch_steps, train_epochs, eval_steps = get_num_train_iterations(
        flags_obj)
    steps_per_loop = min(flags_obj.steps_per_loop, per_epoch_steps)
    train_steps = train_epochs * per_epoch_steps

    logging.info(
        'Training %d epochs, each epoch has %d steps, '
        'total steps: %d; Eval %d steps', train_epochs, per_epoch_steps,
        train_steps, eval_steps)
    
    time_callback = keras_utils.TimeHistory(
        batch_size,
        flags_obj.log_steps,
        summary_writer=train_writer,
        batch_size_per_node=flags_obj.batch_size)
    
    profiler_callback = None
    if flags_obj.profile_steps is not None:
        profiler_callback = keras_utils.get_profiler_callback(
            model_dir,
            flags_obj.profile_steps,
            flags_obj.enable_tensorboard,
            per_epoch_steps)
    with distribution_utils.get_strategy_scope(strategy):
        runnable = resnet_runnable.ResnetRunnable(flags_obj, time_callback,
                                                  train_steps,
                                                  per_epoch_steps,
                                                  profiler_callback)

    eval_interval = flags_obj.epochs_between_evals * per_epoch_steps
    checkpoint_interval = (
        per_epoch_steps if flags_obj.enable_checkpoint_and_export else None)
    summary_interval = per_epoch_steps if flags_obj.enable_tensorboard else None

    checkpoint_manager = tf.train.CheckpointManager(
        runnable.checkpoint,
        directory=model_dir,
        max_to_keep=10,
        step_counter=runnable.global_step,
        checkpoint_interval=checkpoint_interval)

    train_steps=per_epoch_steps * train_epochs

    resnet_controller = controller.Controller(
        strategy,
        runnable.train,
        runnable.evaluate,
        global_step=runnable.global_step,
        steps_per_loop=steps_per_loop,
        train_steps=train_steps,
        checkpoint_manager=checkpoint_manager,
        summary_interval=summary_interval,
        eval_steps=eval_steps,
        eval_interval=eval_interval)

    time_callback.on_train_begin()
    resnet_controller.train(evaluate=not flags_obj.skip_eval)
    time_callback.on_train_end()

    stats = build_stats(runnable, time_callback)
    return stats


After we defined all the functions needed for ResNet50 training, let's start running the workload.

First of all, we will parse `resnet50_keras_lars_bf16_1card.yaml` configuration file to get the input arguments.

In [ ]:
env_args, cmd_args = parse_args_yaml_config('resnet50_keras_lars_bf16_1card.yaml')

cmd_args.insert(0, ' ') # workaround cmd line args 
argv = flags.FLAGS(cmd_args)

Then we will initialize preloading libraries and Synapse logger API

In [ ]:
common.initialize_preloading()
# initialize synapse logger
synapse_logger_init()

Load Habana TensorFlow modules and aquire Habana Gaudi device

In [ ]:
log_info_devices = load_habana_module()
logging.info('Devices:\n%s', log_info_devices)

Now, let's launch ResNet50 training with LARS optimizer on HPU.

In [ ]:
if flags.FLAGS.global_seed:
    tf.random.set_seed(flags.FLAGS.global_seed)

with HabanaEnvVariables(env_args):
    with dump_callback():
        model_helpers.apply_clean(flags.FLAGS)
        with logger.benchmark_context(flags.FLAGS):
            stats =run(flags.FLAGS)
        logging.info('Run stats:\n%s', stats)

To check the training results, we will load TensorBoard extension to this Jupyter Notebook and display the results in charts.

**Note**, if you are running Jupyter Notebook on a web browser on your local machine, you need to forward TensorBoard port number (default is 6006) to the Jupyter Notebook server. The command for port forward looks like this: `ssh -i "dl1-key.pem" -N -f -L localhost:6006:localhost:6006 ubuntu@ec2-35-173-178-212.compute-1.amazonaws.com`

In [ ]:
%load_ext tensorboard

The following command assumes tfevents were dumped to `./model_tmp` directory as specified in `--model_dir` argument in the `resnet50_keras_lars_bf16_1card.yaml` configuration file. Modify it accordingly if you use a different folder as `--model_dir`.

In [ ]:
%tensorboard --bind_all --logdir model_tmp